The code below sends a common push notification to a list of users in a firestore database. It uses Expo server SDK to send the push tokens. It assumes the existence of a firestore database that stores Expo push tokens assigned to users. The code is made of three functions:
1)send_push: This serves as the main (entry point) function. It orchestrates the process by calling the other two.
2)get_tokens: This retrives a list of push tokens from the firestore database
3)send_push_message: This sends the push message to the users

The code can be adjusted to accomodate personalised push messages e.g. Include the name of the user, logic to determine who/who not to send push notification to. This can be done by retrieving other user details from the database and then adding the neccessary logic

In [ ]:
!pip install exponent_server_sdk
!pip install --upgrade firebase-admin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 6.9 MB/s 
     |████████████████████████████████| 106 kB 52.4 MB/s 
     |████████████████████████████████| 244 kB 56.1 MB/s 
     |████████████████████████████████| 244 kB 44.7 MB/s 
     |████████████████████████████████| 1.0 MB 50.1 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 76 kB 4.1 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Attempting uninstall

In [ ]:
#LIGHTAPP

def send_push():
  #send push notifications
  tokens_ = get_tokens()
  for token in tokens_:
    send_push_message(token,'My message to you', extra= {"":""})

def get_tokens():
  '''submits push notifications to users stored in a firestore database
  '''
  #initialise firebase
  import firebase_admin
  from firebase_admin import credentials
  from firebase_admin import firestore

  # Use a service account
  cred = credentials.Certificate('/content/shopmob-secret.json')
  try:
    firebase_admin.initialize_app(cred)
  except:
    print('error occurred in initialising firebase. Perhaps already initialised')

  db = firestore.client()
  #get subscribers list
  users_ref = db.collection(u'light_users')
  docs = users_ref.stream()

  uid_ = []
  secret_ = []
  tokens_ = []

  for doc in docs:
    #print(f'{doc.id} => {doc.to_dict()}')
    id = doc.id
    doc = doc.to_dict()
    #print(doc['email'])
    if doc['status'] == "active":
      tokens_.append(doc['token'])

  return tokens_



# Basic arguments. You should extend this function with the push features you
# want to use, or simply pass in a `PushMessage` object.
def send_push_message(token, message, extra=None):
  from exponent_server_sdk import (
    DeviceNotRegisteredError,
    PushClient,
    PushMessage,
    PushServerError,
    PushTicketError,
  )
  from requests.exceptions import ConnectionError, HTTPError
  try:
      response = PushClient().publish(
          PushMessage(to=token,
                      body=message,
                      data=extra))
      print(response)
  except PushServerError as exc:
      # Encountered some likely formatting/validation error.
      rollbar.report_exc_info(
          extra_data={
              'token': token,
              'message': message,
              'extra': extra,
              'errors': exc.errors,
              'response_data': exc.response_data,
          })
      raise
  except (ConnectionError, HTTPError) as exc:
      # Encountered some Connection or HTTP error - retry a few times in
      # case it is transient.
      rollbar.report_exc_info(
          extra_data={'token': token, 'message': message, 'extra': extra})
      raise self.retry(exc=exc)

  try:
      # We got a response back, but we don't know whether it's an error yet.
      # This call raises errors so we can handle them with normal exception
      # flows.
      response.validate_response()
  except DeviceNotRegisteredError:
      # Mark the push token as inactive
      from notifications.models import PushToken
      PushToken.objects.filter(token=token).update(active=False)
  except PushTicketError as exc:
      # Encountered some other per-notification error.
      rollbar.report_exc_info(
          extra_data={
              'token': token,
              'message': message,
              'extra': extra,
              'push_response': exc.push_response._asdict(),
          })
      raise self.retry(exc=exc)




datetime.datetime(2022, 7, 20, 17, 38, 18, 38229)